In [ ]:
import pypsa
import yaml
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import cartopy.crs as ccrs
import cartopy

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from matplotlib.patches import Circle, Patch
from matplotlib.legend_handler import HandlerPatch

from pypsa.descriptors import get_switchable_as_dense as as_dense

from shapely import wkt
import sys, os

PATH = "../../../../"
SCRIPTS_PATH = "pypsa-eur-sec/scripts/"

sys.path.append(os.path.join(PATH, SCRIPTS_PATH))
# from plot_network import make_legend_circles_for, assign_location
# from helper import override_component_attrs
# from build_gas_input_locations import build_gas_input_locations, load_bus_regions

#plt.style.use(["bmh", "matplotlibrc"])
xr.set_options(display_style="html")

%matplotlib inline

In [ ]:
OUTPUT='/mnt/c/Users/scl38887/Documents/git/aldehyde/results/graphics/first_results/'

#fn = f"{PATH}resources/bus_regions/regions_onshore_elec_s_4.geojson"
fn = '/mnt/c/Users/scl38887/Documents/git/pypsa-earth/resources/bus_regions/regions_onshore_elec_s_4.geojson'
nodes = gpd.read_file(fn).set_index("name")

In [ ]:
# Define the type of spatial plots that are possible with the function "plot_spatial"
label = {"capacity_factor": "Capacity factors in %",
        "capacity_optimised": "Optimised capacity in GW",
        "capacity_potential": "Potentials in GW",
        "land_use": "Land use of available potential in %",
        }
name = {"capacity_factor": "cf",
        "capacity_optimised": "p-nom-opt",
        "capacity_potential": "p-nom-max",
        "land_use": "land-use",
        }
network_names = {"MAR-ES-only": "../../resources/mar_es_only/elec_s_4_ec_lc1.0_Co2L_144H_2030_0.071_DF_nosmr.nc",
                "MAR-ES-export": "../../resources/mar_es_export/elec_s_4_ec_lc1.0_Co2L_144H_2030_0.071_DF_nosmr.nc",
                "Export-only": "../../resources/export_only/elec_s_4_ec_lc1.0_Co2L_144H_2030_0.071_DF_solved.nc",
                "Export-only-high": "../../resources/export_only/elec_s_4_ec_lc1.0_Co2L_144H_2030_0.071_DF_solved_150000000.nc",
                "MAR-ES-export_new": "../../resources/pypsaearthsec/postnetworks/elec_s_4_ec_lc1.0_Co2L_144H_2030_0.071_DF_140export.nc"
                }

In [ ]:
# Import the network 
network = "MAR-ES-export_new"
n = pypsa.Network(network_names[network])
print(f"The network representing {network} has been imported")

### Data preparation

In [ ]:
# Insert hack to copy index of all rows of n.buses with carrier "AC" to the column "location" and avoid length mismatch. They were missing
n.buses.loc[n.buses.carrier == "AC", "location"] = n.buses.index[n.buses.carrier == "AC"]

In [ ]:
techs = [
    #"offwind-ac",
    #"offwind-dc",
    "onwind",
    "solar",
    #"solar thermal",
    #"ror",
    #"air-sourced heat pump",
    #"ground-sourced heat pump",
    #"residential rural solar thermal",
]

### Function definition

In [ ]:
def plot_spatial(
    type,
    df,
    geodf,
    carrier,
    cmap="Blues",
    vmax=100,
    vmin=0,
    #label="capacity factors [%]",
    fn=None,
):
    label_type = label[type]

    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmax=vmax,
        vmin=vmin,
        legend_kwds={
            "label": label_type,
            "shrink": 0.7,
            # "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    #plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor("white")

    if fn is None:
        plt.savefig(f"{OUTPUT}/{network}-{name[type]}-{carrier}.pdf", bbox_inches="tight")

### Capacity Factors Renewables

In [ ]:
df = (
    n.generators_t.p_max_pu.mean()
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .mul(100)
)
df = df[techs]

In [ ]:
plot_spatial("capacity_factor", df, nodes, "onwind", cmap="Blues", vmax=55, vmin=0)

In [ ]:
plot_spatial("capacity_factor", df, nodes, "solar", cmap="Oranges", vmax=max(df["solar"]), vmin=0)

### Potentials Renewables

In [ ]:
df = (
    n.generators.p_nom_max
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .div(1e3) # in GW
)

In [ ]:
plot_spatial("capacity_potential", df, nodes, "onwind", cmap="Blues")

In [ ]:
plot_spatial("capacity_potential", df, nodes, "solar", cmap="Oranges", vmax=max(df["solar"]))

### Optimised Capacity Renewables

In [ ]:
df = (
    n.generators.p_nom_opt
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .div(1e3) # in GW
)

In [ ]:
plot_spatial("capacity_optimised", df, nodes, "onwind", vmax=max(df["onwind"]))

In [ ]:
plot_spatial("capacity_optimised", df, nodes, "solar", "Oranges", vmax=max(df["solar"]))

### Usage Renewables

In [ ]:
df = (
    (n.generators.p_nom_opt/n.generators.p_nom_max)
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .mul(1e2) # in %
)

In [ ]:
plot_spatial("land_use", df, nodes, "onwind", vmax=100)

In [ ]:
plot_spatial("land_use", df, nodes, "solar", "Oranges", vmax=100)